In [2]:
#TO RE-RUN
%reset -f

In [3]:
from sklearn import preprocessing
from time import time
import numpy as np
import csv
from sklearn import metrics
from sklearn.preprocessing import scale
from sklearn.feature_selection import VarianceThreshold
from sklearn.cross_validation import StratifiedShuffleSplit, cross_val_score

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB

from sklearn.grid_search import GridSearchCV, ParameterGrid
from sklearn.preprocessing import StandardScaler

from imblearn.over_sampling import SMOTE,ADASYN, RandomOverSampler
from imblearn.pipeline import Pipeline
from imblearn.pipeline import make_pipeline

from operator import truediv
from datetime import datetime
import pandas as pd
import time
import os

from pylab import *
import seaborn as sns
import matplotlib.pyplot as plt


np.set_printoptions(suppress=True)
pd.options.display.float_format = '{:,.2f}'.format
plt.style.use('classic')

%matplotlib inline

import sys
sys.path.insert(1, "../src/")
from TypeFeatImputer import TypeFeatImputer
from UnivCombineFilter import UnivCombineFilter
import MLpipeline as MLpipeline

/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


#### Run experiments

In [4]:
typeEncounter = "last" # ['first','last']
typeHypothesis = "all_readmisssion_vs_none" # ['all_readmisssion_vs_none','early_readmission_vs_none']
typeDataFeatures = "extended" # ["minimum","extended']
    #Extended -> Subset of columns
    #Minimum -> minimum set of columns 
typeDiagnosis = "none"  #["none","diag_1", "diag_3"]    
typeDisease = "subset" # ["subset","any",["Respiratory",...]]
    #subset -> Return subset of predefined disease features
    #any -> Return all disease features    
    #disease -> Return diseases feature
typeDataExperiment = "disease" #["all", "disease"] 
    #all -> Include all diagnosis as columns
    #disease -> Remove diagnosis as column and keep only rows with diagnosis == 1       

In [5]:
verbose = True
cv_thr = 0.3
cv_folds = 5

tr_thrs = [0.2] # [0.2,0.4,0.6,1.0]
ts_thr = 0.30

fs_methods = ["none"] #["none","combine_fs","lasso_fs","rfe_rf_fs"]
cls_methods = ["rf"] #["rf","svmRBF","logReg","knn","nn"]
lms = ["recall_micro","f1_weighted"] #["f1_weighted","precision_weighted","roc_auc","recall]
sm_types = ["none"] #["none","after"]
sm_method = "sm_smote"

In [6]:
#Load data
df_all = MLpipeline.load_data(typeEncounter, typeDiagnosis, typeDataFeatures)
print "\nSHAPE:"
print df_all.shape
print "\nInitial columns:"
print df_all.columns

#Filter data by class
df_all = MLpipeline.filter_data_by_class(df_all, typeHypothesis)
print "\nRows by class type:"
print df_all.iloc[:,-1].sort_values().unique(), np.sum(df_all["readmitted"] == 0), np.sum(df_all["readmitted"] == 1)
    
#Get columns
colsDiseases, colsNonDiseases = MLpipeline.get_columns(df_all,typeDiagnosis)
print "\nDiseases:", colsDiseases
print "\nNon-diseases:", colsNonDiseases
    
#Load diseases
diseases = MLpipeline.get_diseases(colsDiseases, typeDisease)
print "\nTotal data:", df_all.shape
print diseases


SHAPE:
(67182, 48)

Initial columns:
Index([u'gender', u'age', u'race_AfricanAmerican', u'race_Caucasian',
       u'race_Other', u'HbA1c', u'Change', u'time_in_hospital',
       u'medSpec_cardio', u'medSpec_Family/GeneralPractice',
       u'medSpec_InternalMedicine', u'medSpec_surgery', u'number_treatment',
       u'adm_src_1', u'adm_src_2', u'adm_src_3', u'adm_src_4', u'adm_src_5',
       u'adm_src_6', u'adm_src_7', u'adm_src_8', u'adm_src_10', u'adm_src_11',
       u'adm_src_13', u'adm_src_14', u'adm_src_22', u'adm_src_25', u'adm_1',
       u'adm_2', u'adm_3', u'adm_4', u'adm_7', u'diss_home',
       u'num_lab_procedures', u'num_procedures', u'num_medications',
       u'number_outpatient', u'number_emergency', u'number_inpatient',
       u'number_diagnoses', u'insulin', u'metformin', u'pioglitazone',
       u'glimepiride', u'glipizide', u'repaglinide', u'nateglinide',
       u'readmitted'],
      dtype='object')

Rows by class type:
[0 1] 39785 27397


../src/MLpipeline.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_all["readmitted"][df_all["readmitted"].values > 0] = 1


UnboundLocalError: local variable 'colsDiseases' referenced before assignment

In [6]:
res = MLpipeline.run(df_all, colsNonDiseases, diseases, typeDataFeatures, 
               typeDataExperiment, typeEncounter, typeHypothesis, typeDiagnosis, ts_thr, tr_thrs, 
               fs_methods, sm_method, sm_types, 
               cls_methods, lms, cv_folds, cv_thr, True, False)

(2, 6)

DataSet:
**********
**********
SIZE: 1.0
DISEASE: Neoplasms_1
(2718, 22)
ALL TRAIN: (1902, 21)
TRAIN: [0's: 1327 1's: 575 ]
ALL TEST: (816, 21)
TEST: [0's: 570 1's: 246 ]

Num experiment: 0 / 1
****************
FS: none
SM: none
CLS: rf
METRIC: recall_micro
Fitting 5 folds for each of 60 candidates, totalling 300 fits


/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/utils/validation.py:444: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/home/ilmira/.conda/envs/readmision/lib/python2.7/si


CV INNER metric: recall_micro
CV INNER selected params ['gini', 6, None, 100]
CV INNER score: 0.702276707531

CV OUTER f1 score: 0.587  (+/-0.005)
CV OUTER prec score: 0.661  (+/-0.056)
CV OUTER rec score: 0.698  (+/-0.005)
Selected params (bests from CV) ['gini', 6, None, 100]

TR F1 score: 0.62868448227
TR Prec score: 0.801682382821
TR Rec score: 0.722923238696

Test f1 (weighted): 0.583
Test Precision (weighted): 0.697
Test Recall / Sensitivity (weighted): 0.623
Test AUC (weighted): 0.504
Test Sensitivity: 0.016
Test Specificity: 0.991
Test AUC: 0.504

with following performance in test:
             precision    recall  f1-score   support

          0       0.70      0.99      0.82       570
          1       0.44      0.02      0.03       246

avg / total       0.62      0.70      0.58       816

[[565   5]
 [242   4]]

Total time: 57.2521739006

Num experiment: 1 / 1
****************
FS: none
SM: none
CLS: rf
METRIC: f1_weighted
Fitting 5 folds for each of 60 candidates, totalli

/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.8s
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ilmira/.conda/envs/readmision/lib/python2.7/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.


CV INNER metric: f1_weighted
CV INNER selected params ['gini', 6, 'balanced', 200]
CV INNER score: 0.662973278309

CV OUTER f1 score: 0.652  (+/-0.015)
CV OUTER prec score: 0.653  (+/-0.014)
CV OUTER rec score: 0.650  (+/-0.016)
Selected params (bests from CV) ['gini', 6, 'balanced', 200]

TR F1 score: 0.783485759437
TR Prec score: 0.790159067349
TR Rec score: 0.779705573081

Test f1 (weighted): 0.613
Test Precision (weighted): 0.605
Test Recall / Sensitivity (weighted): 0.623
Test AUC (weighted): 0.555
Test Sensitivity: 0.427
Test Specificity: 0.682
Test AUC: 0.555

with following performance in test:
             precision    recall  f1-score   support

          0       0.73      0.68      0.71       570
          1       0.37      0.43      0.39       246

avg / total       0.62      0.61      0.61       816

[[389 181]
 [141 105]]

Total time: 59.6393430233


In [7]:
if 'dfAux' not in locals():
    print "Not dfAux"
    dfAux = pd.DataFrame()

if 'df' in locals():
    print "Exist df"
    dfAux = df.copy()
    
df = pd.DataFrame(np.array(res).reshape(len(res),27), columns=
                          ["exp", "typeDisease","typeEncounter","typeHypothesis","typeDataFeatures","typeDiagnosis",
                           "size_tr","fs","sm","cls","metric","params",
                           "tr_f1","tr_prec","tr_rec",
                           "cv_f1_mean","cv_f1_std","cv_prec_mean","cv_prec_std","cv_rec_mean","cv_rec_std",
                           "test_f1","test_prec","test_rec","test_auc",
                           "time","pipeline"])

df[["size_tr","sm","metric","cls","params", 'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 'test_f1', 'test_prec', 'test_rec', 'test_auc']]

Not dfAux


,size_tr,sm,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
0,1.00,none,recall_micro,rf,"[gini, 6, None, 100]",0.63,0.80,0.72,0.59,0.66,0.70,0.58,0.70,0.62,0.50
1,1.00,none,f1_weighted,rf,"[gini, 6, balanced, 200]",0.78,0.79,0.78,0.65,0.65,0.65,0.61,0.61,0.62,0.55


In [8]:
df = pd.concat((dfAux, df))
df[["size_tr","sm","metric","cls","params",'tr_f1', 'tr_prec', 'tr_rec', 
     'cv_f1_mean', 'cv_prec_mean', 'cv_rec_mean', 'test_f1', 'test_prec', 
        'test_rec', 'test_auc']]

,size_tr,sm,metric,cls,params,tr_f1,tr_prec,tr_rec,cv_f1_mean,cv_prec_mean,cv_rec_mean,test_f1,test_prec,test_rec,test_auc
0,1.00,none,recall_micro,rf,"[gini, 6, None, 100]",0.63,0.80,0.72,0.59,0.66,0.70,0.58,0.70,0.62,0.50
1,1.00,none,f1_weighted,rf,"[gini, 6, balanced, 200]",0.78,0.79,0.78,0.65,0.65,0.65,0.61,0.61,0.62,0.55
